# 🌍 duckOSM Network Explorer

This notebook allows you to interactively explore and visualize the routing networks generated by `duckOSM`.

In [ ]:
import duckdb
import leafmap.foliumap as leafmap
import geopandas as gpd
from shapely import wkt
import os

## 1. Connect to Database
Select your transportation mode (e.g., `driving`, `walking`, `cycling`).

In [ ]:
db_path = '../data/output/somerset.duckdb'
mode = 'driving'

con = duckdb.connect(db_path)
con.execute("INSTALL spatial; LOAD spatial;")

# Check available schemas
con.execute("SELECT schema_name FROM information_schema.schemata WHERE schema_name NOT IN ('information_schema', 'pg_catalog', 'main', 'raw')").fetchall()

## 2. Load Network Data
We load the edges into a GeoDataFrame for visualization. Note that we use `ST_AsText(geometry)` because we are now using native DuckDB spatial types.

In [ ]:
query = f"""
    SELECT 
        edge_id, osm_id, name, highway, 
        length_m, maxspeed_kmh, cost_s,
        ST_AsText(geometry) as geometry
    FROM {mode}.edges
    LIMIT 5000
"""
df = con.execute(query).df()

# Convert WKT to geometry
df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326")

gdf.head()

## 3. Visualize on Map
Click on any edge to see its properties.

In [ ]:
m = leafmap.Map(center=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom=13)

# Simple coloring based on highway type
m.add_gdf(
    gdf, 
    layer_name=f"{mode.capitalize()} Network",
    info_mode="on_click"
)
m